Robot Localization with Python and Particle Filters
===================================================

Import libraries and load map.

In [1]:
import numpy as np
import cv2
map = cv2.imread("map.png",0)
HEIGHT,WIDTH = map.shape
print(map)
rx,ry,rtheta = (WIDTH/4,HEIGHT/4,0)

[[ 79  80  82 ... 133 148 156]
 [ 78  79  81 ... 138 156 164]
 [ 76  77  79 ... 147 170 180]
 ...
 [181 181 182 ... 174 172 171]
 [178 178 178 ... 180 179 178]
 [177 177 177 ... 183 182 182]]


Map coordinate system

![title](images/coords.png)

CAUTION: The terrain height at X,Y coordinates is map(Y,X).

Read keyboard input.

In [2]:
STEP = 5
TURN =np.radians(25)
def get_input():
    fwd = 0
    turn =  0
    halt = False
    k = cv2.waitKey(0)
    if k == 82:  # Up arrow
        fwd = STEP
    elif k == 83: # Right arrow
        turn = TURN
    elif k == 81: # Left arrow
        turn = -TURN
    elif k == 27:  # ESC to halt 
        halt = True
    return fwd, turn, halt

Move the robot, with Gausssian noise.

![title](images/gaussian.png)

In [3]:
SIGMA_STEP = 0.5
SIGMA_TURN = np.radians(5)

def move_robot(rx, ry, rtheta, fwd, turn):
    fwd_noisy = fwd + np.random.normal(0.0, SIGMA_STEP, 1)
    rx += fwd_noisy * np.cos(rtheta)
    ry += fwd_noisy * np.sin(rtheta)

    turn_noisy = turn + np.random.normal(0.0, SIGMA_TURN, 1)
    rtheta += turn_noisy

    print("turn_noisy=",np.degrees(turn_noisy))
    
    return rx, ry, rtheta

Initialize particle cloud.

In [4]:
NUM_PARTICLES = 3000
def init():
    particles = np.random.rand(NUM_PARTICLES,3)
    particles *=np.array( (WIDTH,HEIGHT,np.radians(360)))

    return particles

Move the particles.

In [5]:
def move_particles(particles, fwd, turn):
    particles[:,0] += fwd * np.cos(particles[:,2])
    particles[:,1] += fwd * np.sin(particles[:,2])
    particles[:,2] += turn

    particles[:,0] = np.clip(particles[:,0], 0.0, WIDTH-1)
    particles[:,1] = np.clip(particles[:,1], 0.0, HEIGHT-1)
    return particles
    return particles

Get value from robot's sensor.

In [6]:
SIGMA_SENSOR = 2
def sense(x, y, noisy=False):
    x = int(x)
    y = int(y)
    if noisy:
        return np.random.normal(map[y,x], SIGMA_SENSOR, 1)
    return map[y,x]

Compute particle weights.

In [7]:
def compute_weights(particles, robot_sensor):    
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        elevation = sense(particles[i,0],particles[i,1], noisy =False)
        errors[i] = abs(robot_sensor - elevation)
    weights = np.max(errors) - errors
    weights[
    (particles[:,0] ==0) |
    (particles[:,0] == WIDTH-1) |
    (particles[:,1] ==0) |
    (particles[:,1] == HEIGHT-1) 
    ] = 0.0
    
    weights = weights **3
    return weights

Resample the particles.

In [8]:
def resample(particles, weights):
    probabilities = weights / np.sum(weights)
    new_index = np.random.choice(
    NUM_PARTICLES,
    size = NUM_PARTICLES,
    p = probabilities
    )
    particles = particles[new_index,:]
    return particles

Add noise to the particles.

In [9]:
SIGMA_POS =2
SIGMA_TURN = np.radians(10)

def add_noise(particles):
    noise = np.concatenate(
        (
            np.random.normal(0, SIGMA_POS,(NUM_PARTICLES,1)),
            np.random.normal(0, SIGMA_POS,(NUM_PARTICLES,1)),
            np.random.normal(0, SIGMA_TURN,(NUM_PARTICLES,1)),
        ),
        axis =1 
    )
    particles += noise
    return particles

Display robot, particles and best guess.

In [10]:
def display(map, rx, ry, particles):
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)
    
    # Display particles
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            cv2.circle(lmap, 
                       (int(particles[i,0]), int(particles[i,1])), 
                       1, 
                       (255,0,0), 
                       1)
        
    # Display robot
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0,255,0), 10)

    # Display best guess
    if len(particles) > 0 :
        px = np.mean(particles[:,0])
        py = np.mean(particles[:,1])
        cv2.circle(lmap, (int(px), int(py)), 5, (0,0,255), 5)

    cv2.imshow('map', lmap)

Main routine.

In [11]:
particles = init()
while True:
    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)        
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)
    
cv2.destroyAllWindows()                        


turn_noisy= [0.27444079]
turn_noisy= [21.75502037]
turn_noisy= [-1.86299081]
turn_noisy= [12.92050177]
turn_noisy= [5.74751827]
turn_noisy= [-17.10757205]
turn_noisy= [3.94192116]
turn_noisy= [11.6170215]
turn_noisy= [-0.97105081]
turn_noisy= [3.95066535]
turn_noisy= [-5.64996378]
turn_noisy= [-8.10815462]
turn_noisy= [15.77129259]
turn_noisy= [3.45426278]
turn_noisy= [7.97310303]
turn_noisy= [27.67225289]
turn_noisy= [5.20808983]
turn_noisy= [0.83124126]
turn_noisy= [7.99763791]
turn_noisy= [5.89849501]
turn_noisy= [6.36732232]
turn_noisy= [-3.25358003]
turn_noisy= [0.34273249]
turn_noisy= [-9.35795889]
turn_noisy= [0.80177778]
turn_noisy= [14.59321847]
turn_noisy= [-22.29263481]
turn_noisy= [8.61364032]
turn_noisy= [-15.16405682]
turn_noisy= [-1.50643176]
turn_noisy= [7.3198028]
turn_noisy= [-13.50202957]
turn_noisy= [-4.1843352]
turn_noisy= [-4.34973255]
turn_noisy= [-15.13753439]
turn_noisy= [2.26150846]
turn_noisy= [-15.3842056]
turn_noisy= [1.95918007]
turn_noisy= [1.83077575]
tu